In [169]:
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

# 1. Obtention des données
## Chargement des fichiers
On charge ici tous les fichiers de l'export (préalablement compressés pour préserver la place sur le disque)

In [170]:
analyses_orthophosphate = pd.read_csv("./naiades_export_orthophosphate/Analyses.CSV",sep=";",parse_dates=["DatePrel"])

In [171]:
pd.set_option('display.max_columns', None)
analyses_orthophosphate.columns


Index(['CdStationMesureEauxSurface', 'LbStationMesureEauxSurface', 'CdSupport',
       'LbSupport', 'CdFractionAnalysee', 'LbFractionAnalysee',
       'CdPrelevement', 'DatePrel', 'HeurePrel', 'DateAna', 'HeureAna',
       'CdParametre', 'LbLongParamètre', 'RsAna', 'CdUniteMesure',
       'SymUniteMesure', 'CdRqAna', 'MnemoRqAna', 'CdInsituAna', 'LbInsituAna',
       'ProfondeurPrel', 'CdDifficulteAna', 'MnemoDifficulteAna', 'LdAna',
       'LqAna', 'LsAna', 'IncertAna', 'CdMetFractionnement',
       'NomMetFractionnement', 'CdMethode', 'NomMethode', 'RdtExtraction',
       'CdMethodeExtraction', 'NomMethodeExtraction', 'CdAccreAna',
       'MnemoAccredAna', 'AgreAna', 'CdStatutAna', 'MnemoStatutAna',
       'CdQualAna', 'LbQualAna', 'CommentairesAna', 'ComResultatAna', 'CdRdd',
       'NomRdd', 'CdProducteur', 'NomProducteur', 'CdPreleveur',
       'NomPreleveur', 'CdLaboratoire', 'NomLaboratoire'],
      dtype='object')

In [172]:
analyses_orthophosphate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9729 entries, 0 to 9728
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   CdStationMesureEauxSurface  9729 non-null   int64         
 1   LbStationMesureEauxSurface  9729 non-null   object        
 2   CdSupport                   9729 non-null   int64         
 3   LbSupport                   9729 non-null   object        
 4   CdFractionAnalysee          9729 non-null   int64         
 5   LbFractionAnalysee          9729 non-null   object        
 6   CdPrelevement               9729 non-null   int64         
 7   DatePrel                    9729 non-null   datetime64[ns]
 8   HeurePrel                   9724 non-null   object        
 9   DateAna                     9710 non-null   object        
 10  HeureAna                    9524 non-null   object        
 11  CdParametre                 9729 non-null   int64       

In [173]:
analyses_orthophosphate = analyses_orthophosphate[pd.to_datetime(analyses_orthophosphate['DatePrel']).dt.year != 2022]
analyses_orthophosphate = analyses_orthophosphate[pd.to_datetime(analyses_orthophosphate['DatePrel']).dt.year != 2023]

In [174]:
new_df = analyses_orthophosphate.loc[:, ['CdStationMesureEauxSurface', 'CdPrelevement', 'RsAna', 'DatePrel']]
print(new_df)


      CdStationMesureEauxSurface  CdPrelevement  RsAna   DatePrel
0                        2000003        6994353  0.430 2020-02-03
1                        2000003        7027672  0.353 2020-05-25
2                        2000003        7031837  1.470 2020-07-20
3                        2000003        7035763  9.497 2020-09-14
4                        2000003        7037335  4.601 2020-11-09
...                          ...            ...    ...        ...
9708                     6455540         740681  0.150 2019-03-06
9725                     6455540         823738  0.430 2020-12-02
9726                     6455540         823740  0.640 2020-06-04
9727                     6455540         823929  0.190 2020-03-05
9728                     6455540         823930  0.360 2020-09-09

[9658 rows x 4 columns]


In [175]:
new_df['Month'] = pd.to_datetime(new_df['DatePrel']).dt.month
new_df['Year'] = pd.to_datetime(new_df['DatePrel']).dt.year

new_df = new_df.dropna(subset=['RsAna'])
preprocess_df = new_df.drop(columns=['CdPrelevement', 'DatePrel'])
# Remove stations with less than 15 rows


In [176]:

df = px.data.tips()
fig = px.box(preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False))
fig.show()
station_counts = preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False)
station_counts

CdStationMesureEauxSurface
2001700    78
2001050    78
2048980    40
2094973    40
2052500    40
           ..
2044400     1
2100940     1
2098090     1
2098450     1
2001046     1
Length: 395, dtype: int64

In [177]:

preprocess_df = preprocess_df.groupby('CdStationMesureEauxSurface').filter(lambda x: len(x) > 33 and len(x) <= 50)


In [178]:
station_counts = preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False)
station_counts


CdStationMesureEauxSurface
2044300    40
2095600    40
2094973    40
2096480    40
2048980    40
           ..
2084200    34
2048960    34
2043775    34
2036265    34
2000010    34
Length: 206, dtype: int64

In [179]:
df = px.data.tips()
fig = px.box(preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False))
fig.show()